# Álgebra linear em Julia

## Operações básicas

Primeiro, vamos construir uma matriz 3x3 com entradas entre 1 e 4 escolhidas aleatoriamente:

In [1]:
A = rand(1:4, 3, 3)

3×3 Matrix{Int64}:
 4  4  4
 4  3  3
 3  2  3

Agora, vamos construir um vetor com 3 coordenadas todas iguais a 1:

In [2]:
v = fill(1.0, (3))

3-element Vector{Float64}:
 1.0
 1.0
 1.0

### Multiplicação

In [3]:
b = A*v

3-element Vector{Float64}:
 12.0
 10.0
  8.0

### Transposição

A matriz `transpose(A)` é a transposta de `A`:

In [4]:
transpose(A)

3×3 transpose(::Matrix{Int64}) with eltype Int64:
 4  4  3
 4  3  2
 4  3  3

A matriz `A'` é a adjunta de `A`, ou seja, a transposta conjugada de `A`, que coincide com a transposta quando `A` é real.

In [5]:
A'

3×3 adjoint(::Matrix{Int64}) with eltype Int64:
 4  4  3
 4  3  2
 4  3  3

### Multiplicação pela transposta

Em Julia, podemos escrever simplesmente `A'A` em vez de `(A')*A`:

In [6]:
A'A

3×3 Matrix{Int64}:
 41  34  37
 34  29  31
 37  31  34

In [7]:
(A')*A

3×3 Matrix{Int64}:
 41  34  37
 34  29  31
 37  31  34

## Resolvendo sistemas de equações lineares

O problema $Ax = b$ pode ser resolvido usando a função `\`.

In [8]:
x = A\b

3-element Vector{Float64}:
 1.0
 1.0
 1.0

## Matrizes com estrutura especial

In [9]:
using LinearAlgebra

In [10]:
n = 1000
A = randn(n,n);

Frequentemente, é possível inferir a estrutura de uma matriz:

In [11]:
Asim = A + A'
issymmetric(Asim)

true

Todavia, muitas vezes os arredondamentos numéricos atrapalham:

In [12]:
Asim_ruido = copy(Asim)
ruido = 5eps()
println("ruido = $ruido")
Asim_ruido[1,2] += ruido
issymmetric(Asim_ruido)

ruido = 1.1102230246251565e-15


false

O comando acima inseriu um ruído da ordem de 10<sup>-15</sup> no elemento (1,2) da matriz `Asim`.

A linguagem Julia permite que declaremos explicitamente a estrutura de uma matriz usando, por exemplo, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` e `SymTridiagonal`.

In [13]:
Asim_explicita = Symmetric(Asim_ruido)
issymmetric(Asim_explicita)

true

Vamos comparar quanto tempo leva para a Julia calcular os autovalores de `Asim`, `Asim_ruido` e `Asim_explicita`.

In [14]:
@time eigvals(Asim);

  0.408395 seconds (668.78 k allocations: 46.791 MiB, 2.21% gc time, 83.75% compilation time)


In [15]:
@time eigvals(Asim_ruido);

  0.630018 seconds (13 allocations: 7.920 MiB)


In [16]:
@time eigvals(Asim_explicita);

  0.083102 seconds (5.88 k allocations: 8.312 MiB, 9.85% gc time, 13.98% compilation time)


Observamos que o cálculo com `Asim_explicita` (onde usamos `Symmetric()`) foi muitas vezes mais rápido do que o cálculo com `Asim_ruido`.

## Um problema grande

Usando os tipos `Tridiagonal` e `SymTridiagonal`, podemos armazenar matrizes de forma eficiente.
Isso torna possível fazer cálculos com matrizes muito grandes.
Por exemplo, não seria possível resolver em um laptop o seguinte problema se armazenássemos a matriz usando o tipo `Array`.

In [17]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.742524 seconds (835.74 k allocations: 199.522 MiB, 20.14% gc time, 30.15% compilation time)


6.319141104185533

O cálculo acima demorou menos de 1 segundo.
Explicamos a seguir o significado dos comandos.

O comando abaixo cria uma matriz simétrica 6x6 especificando 6 números aleatórios na diagonal e 5 números aleatórios na diagonal secundária.

In [18]:
B = SymTridiagonal(randn(6), randn(6-1))

6×6 SymTridiagonal{Float64, Vector{Float64}}:
 -1.2491       0.00426921    ⋅          ⋅          ⋅           ⋅ 
  0.00426921  -1.36801      1.77387     ⋅          ⋅           ⋅ 
   ⋅           1.77387      0.507839  -0.279136    ⋅           ⋅ 
   ⋅            ⋅          -0.279136   1.39393   -0.723164     ⋅ 
   ⋅            ⋅            ⋅        -0.723164   1.32807    -0.0566859
   ⋅            ⋅            ⋅          ⋅        -0.0566859   1.16555

Portanto, a matriz `A` acima tem estrutura semelhante mas é de ordem **1 milhão x 1 milhão**:

In [19]:
A

1000000×1000000 SymTridiagonal{Float64, Vector{Float64}}:
  0.657122  -1.12338    ⋅         ⋅        …    ⋅          ⋅         ⋅ 
 -1.12338   -0.212994  0.281223   ⋅             ⋅          ⋅         ⋅ 
   ⋅         0.281223  0.473275  1.08835        ⋅          ⋅         ⋅ 
   ⋅          ⋅        1.08835   0.974779       ⋅          ⋅         ⋅ 
   ⋅          ⋅         ⋅        0.387588       ⋅          ⋅         ⋅ 
   ⋅          ⋅         ⋅         ⋅        …    ⋅          ⋅         ⋅ 
   ⋅          ⋅         ⋅         ⋅             ⋅          ⋅         ⋅ 
   ⋅          ⋅         ⋅         ⋅             ⋅          ⋅         ⋅ 
   ⋅          ⋅         ⋅         ⋅             ⋅          ⋅         ⋅ 
   ⋅          ⋅         ⋅         ⋅             ⋅          ⋅         ⋅ 
   ⋅          ⋅         ⋅         ⋅        …    ⋅          ⋅         ⋅ 
   ⋅          ⋅         ⋅         ⋅             ⋅          ⋅         ⋅ 
   ⋅          ⋅         ⋅         ⋅             ⋅          ⋅         ⋅ 
  ⋮   

O comando `randn(6)` cria uma lista de 6 números aleatórios.

In [20]:
randn(6)

6-element Vector{Float64}:
  0.2710472403537215
 -0.009293746568878308
  0.7835409486025717
  0.9271807496403283
  0.07057502284252713
  2.3358150674910876

O comando `eigmax(A)` calcula o maior autovalor de `A`.

Com os comandos a seguir, podemos ler as documentações sobre as funções (lembre da dica de traduzir usando https://translate.google.com):

In [21]:
?eigmax

search: eigmax



```
eigmax(A; permute::Bool=true, scale::Bool=true)
```

Return the largest eigenvalue of `A`. The option `permute=true` permutes the matrix to become closer to upper triangular, and `scale=true` scales the matrix by its diagonal elements to make rows and columns more equal in norm. Note that if the eigenvalues of `A` are complex, this method will fail, since complex numbers cannot be sorted.

# Examples

```jldoctest
julia> A = [0 im; -im 0]
2×2 Matrix{Complex{Int64}}:
 0+0im  0+1im
 0-1im  0+0im

julia> eigmax(A)
1.0

julia> A = [0 im; -1 0]
2×2 Matrix{Complex{Int64}}:
  0+0im  0+1im
 -1+0im  0+0im

julia> eigmax(A)
ERROR: DomainError with Complex{Int64}[0+0im 0+1im; -1+0im 0+0im]:
`A` cannot have complex eigenvalues.
Stacktrace:
[...]
```


In [22]:
?randn

search: randn lowrankdowndate lowrankdowndate! RankDeficientException rand



```
randn([rng=GLOBAL_RNG], [T=Float64], [dims...])
```

Generate a normally-distributed random number of type `T` with mean 0 and standard deviation 1. Optionally generate an array of normally-distributed random numbers. The `Base` module currently provides an implementation for the types [`Float16`](@ref), [`Float32`](@ref), and [`Float64`](@ref) (the default), and their [`Complex`](@ref) counterparts. When the type argument is complex, the values are drawn from the circularly symmetric complex normal distribution of variance 1 (corresponding to real and imaginary part having independent normal distribution with mean zero and variance `1/2`).

# Examples

```jldoctest
julia> using Random

julia> rng = MersenneTwister(1234);

julia> randn(rng, ComplexF64)
0.6133070881429037 - 0.6376291670853887im

julia> randn(rng, ComplexF32, (2, 3))
2×3 Matrix{ComplexF32}:
 -0.349649-0.638457im  0.376756-0.192146im  -0.396334-0.0136413im
  0.611224+1.56403im   0.355204-0.365563im  0.0905552+1.31012im
```
